<a href="https://colab.research.google.com/github/mmura001/Healthbot/blob/main/BBN_dynamic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
from pgmpy.inference import VariableElimination




In [20]:
pip install pgmpy

In [11]:

# def build_bayesian_network(csv_file):
#     # Load data from the CSV file
#     data = pd.read_csv(csv_file)

#     column_names = data.columns.tolist()

#     # Print the list of column names
#     print("Column Names:", column_names)

#     # Create a correlation matrix
#     corr_matrix = data.corr()
#     print(corr_matrix)

#     # Define a Bayesian Model
#     bayesian_model = BayesianModel()

#     # Add nodes to the Bayesian Model
#     bayesian_model.add_nodes_from(corr_matrix.columns)

#     for i in range(len(corr_matrix.columns)):
#         for j in range(i + 1, len(corr_matrix.columns)):
#             correlation_value = corr_matrix.iloc[i, j]
#             if abs(correlation_value) > 0.95 or (0 <= correlation_value <= 0.1):  # Adjust the threshold as needed
#                 bayesian_model.add_edge(corr_matrix.columns[i], corr_matrix.columns[j])
#     model = BayesianModel(bayesian_model.edges())

#     # Estimate CPDs for each node
#     # cpd_TotPkts = MaximumLikelihoodEstimator(model, data).estimate_cpd(column_names[1])
#     # cpd_SrcPkts = MaximumLikelihoodEstimator(model, data).estimate_cpd(column_names[2])
#     # cpd_TotBytes = MaximumLikelihoodEstimator(model, data).estimate_cpd(column_names[3])
#     # cpd_SrcBytes = MaximumLikelihoodEstimator(model, data).estimate_cpd(column_names[4])
#     # cpd_DstPkts = MaximumLikelihoodEstimator(model, data).estimate_cpd(column_names[5])
#     # print("column_names[6]",column_names[6])
#     # print("Nodes in the Bayesian network:", bayesian_model.nodes())
#     # cpd_Target = BayesianEstimator(model, data).estimate_cpd(column_names[6], prior_type="BDeu", equivalent_sample_size=20)

#     # Estimate CPDs for each node
#     cpds = {}
#     for column_name in column_names[1:]:
#         cpds[column_name] = MaximumLikelihoodEstimator(bayesian_model, data).estimate_cpd(column_name)

#     print("Nodes in the Bayesian network:", bayesian_model.nodes())

#     # Add CPDs to the model
#     for column_name, cpd in cpds.items():
#         bayesian_model.add_cpds(cpd)
#         # print("CPD", cpds)

#     # Add CPDs to the model
#     # model.add_cpds(cpd_TotPkts, cpd_SrcPkts, cpd_TotBytes, cpd_SrcBytes, cpd_DstPkts, cpd_Target)

#     # Initialize VariableElimination object for inference
#     bayesian_model.checker = VariableElimination(model)

#     return model


In [90]:
def build_bayesian_network_2(csv_file):
    # Load data from the CSV file
    data = pd.read_csv(csv_file)
    data.drop("Sport", axis=1, inplace=True)

    column_names = data.columns.tolist()

    # Print the list of column names
    print("Column Names:", column_names)

    # Create a correlation matrix
    corr_matrix = data.corr()
    print(corr_matrix)

    # Define a Bayesian Model
    bayesian_model = BayesianModel()

    # Add nodes to the Bayesian Model
    bayesian_model.add_nodes_from(corr_matrix.columns)

    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):
            correlation_value = corr_matrix.iloc[i, j]
            if abs(correlation_value) > 0.95 or (-0.5 <= correlation_value <= 0.1):  # Adjust the threshold as needed
                bayesian_model.add_edge(corr_matrix.columns[i], corr_matrix.columns[j])

    # Estimate CPDs for each node
    n = len(column_names)
    cpds = []
    for column_name in column_names[0:n-1]:
        cpd = MaximumLikelihoodEstimator(bayesian_model, data).estimate_cpd(column_name)
        bayesian_model.add_cpds(cpd)
        print("updated{}, cpd {}".format(column_name,cpd))


    cpd_Target = BayesianEstimator(bayesian_model,data).estimate_cpd(column_names[n-1],prior_type="BDeu", equivalent_sample_size=20)
    bayesian_model.add_cpds(cpd_Target)
    print(cpd_Target)

    # Initialize VariableElimination object for inference
    bayesian_model.checker = VariableElimination(bayesian_model)

    return bayesian_model


In [5]:


def query_bayesian_network(model, query_variables, evidence):
    # Perform inference/query on the Bayesian network model
    result = model.checker.query(variables=query_variables, evidence=evidence)
    return result


In [93]:
csv_file = "Target_Syntheticdata.csv"  # Replace with the path to your CSV file
model = build_bayesian_network_2(csv_file)


Column Names: ['TotPkts', 'TotBytes', 'SrcPkts', 'DstPkts', 'SrcBytes', 'Target']
           TotPkts  TotBytes   SrcPkts   DstPkts  SrcBytes    Target
TotPkts   1.000000 -0.748277  0.708563  0.685533 -0.747515 -0.244532
TotBytes -0.748277  1.000000 -0.938517 -0.962249  0.999874  0.461947
SrcPkts   0.708563 -0.938517  1.000000  0.901224 -0.938294 -0.376677
DstPkts   0.685533 -0.962249  0.901224  1.000000 -0.962729 -0.506359
SrcBytes -0.747515  0.999874 -0.938294 -0.962729  1.000000  0.464954
Target   -0.244532  0.461947 -0.376677 -0.506359  0.464954  1.000000
<class 'list'>
length 6
column_names[1, n-1] TotPkts SrcBytes
['TotPkts', 'TotBytes', 'SrcPkts', 'DstPkts']
updatedTotPkts, cpd +-------------+---------+
| TotPkts(-4) | 0.00275 |
+-------------+---------+
| TotPkts(-3) | 0.31175 |
+-------------+---------+
| TotPkts(-2) | 0.67675 |
+-------------+---------+
| TotPkts(-1) | 0.00875 |
+-------------+---------+
updatedTotBytes, cpd +---------------+---------+
| TotBytes(201) | 0.0002

In [95]:

# Define query variables and evidence
query_variables = ["Target"]
evidence = {"TotPkts": -2, "TotBytes": 290, "SrcPkts": -5, "DstPkts": -7, "SrcBytes": 154}

# Query the Bayesian network model
result = query_bayesian_network(model, query_variables, evidence)
print(result)

+-----------+---------------+
| Target    |   phi(Target) |
+===========+===============+
| Target(0) |        0.7428 |
+-----------+---------------+
| Target(1) |        0.2572 |
+-----------+---------------+
